In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.callbacks import ModelCheckPoint
import matplotlib.pyplot as plt
from keras.preprocessing import image
import cv2
import os

In [3]:
categories=['with Mask','without Mask']
data=[]


In [4]:
for category in categories:
    path=os.path.join('train',category)
    label=categories.index(category)
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])
        #print(img.shape)

In [5]:
len(data)

2646

In [6]:
X=[]
y=[]
for features,label in data:
    X.append(features)
    y.append(label)

In [7]:
import numpy as np
X=np.array(X)
y=np.array(y)

In [8]:
X.shape

(2646, 224, 224, 3)

In [9]:
y.shape

(2646,)

In [10]:
X=X/255

In [11]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [16]:
from keras.applications.vgg16 import VGG16

In [17]:
vgg=VGG16()

In [18]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [52]:
def detect_mask(img):
    #y_pred=model.predict_classes(img.reshape(1,224,224,3))
    predict_x=model_1.predict(img.reshape(1,224,224,3)) 
    class_x=np.argmax(predict_x,axis=1)
    print(predict_x)
    if class_x[0]==0:
        return 'No Mask'
    elif class_x[0]==1:
        return 'Mask detected'

# Initial Model

In [25]:
model=Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [27]:
for layers in model.layers:
    layer.trainable=False

In [28]:
from keras.layers import Dense
model.add(Dense(1,activation='sigmoid'))

In [29]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [21]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
67/67 [==============================] - 1363s 20s/step - loss: 1.0381 - accuracy: 0.4910 - val_loss: 0.6937 - val_accuracy: 0.5057
Epoch 2/5
67/67 [==============================] - 1270s 19s/step - loss: 0.7627 - accuracy: 0.5156 - val_loss: 0.6931 - val_accuracy: 0.5057
Epoch 3/5
67/67 [==============================] - 1207s 18s/step - loss: 0.6940 - accuracy: 0.4934 - val_loss: 0.6931 - val_accuracy: 0.5057
Epoch 4/5
67/67 [==============================] - 1175s 18s/step - loss: 0.6943 - accuracy: 0.5113 - val_loss: 0.6932 - val_accuracy: 0.4943
Epoch 5/5
67/67 [==============================] - 1182s 18s/step - loss: 0.6940 - accuracy: 0.5052 - val_loss: 0.6932 - val_accuracy: 0.4943


In [84]:
def detect_mask(img):
    #y_pred=model.predict_classes(img.reshape(1,224,224,3))
    predict_x=model.predict(img.reshape(1,224,224,3)) 
    class_x=np.argmax(predict_x,axis=1)
    print(predict_x)
    if class_x[0]==0:
        return 'No Mask'
    elif class_x[0]==1:
        return 'Mask detected'

In [88]:
sample1=cv2.imread('sample/1.jpg')
sample1=cv2.resize(sample1,(224,224))
detect_mask(sample1)

[[0.49835473]]


'No Mask'

In [87]:
sample2=cv2.imread('sample/2.jpg')
sample2=cv2.resize(sample1,(224,224))
detect_mask(sample2)

[[0.49835473]]


'No Mask'

In [86]:
for i in range(3,21):
    img='sample/'+str(i)+'.png'
    sample=cv2.imread(img)
    sample=cv2.resize(sample,(224,224))
    
    print(detect_mask(sample))

[[0.4979835]]
No Mask
[[0.4984745]]
No Mask
[[0.50035506]]
No Mask
[[0.49840933]]
No Mask
[[0.4979782]]
No Mask
[[0.4994214]]
No Mask
[[0.49835864]]
No Mask
[[0.49967244]]
No Mask
[[0.4986017]]
No Mask
[[0.4992371]]
No Mask
[[0.4980116]]
No Mask
[[0.4981108]]
No Mask
[[0.4980121]]
No Mask
[[0.49857858]]
No Mask
[[0.49771866]]
No Mask
[[0.49732038]]
No Mask
[[0.49795413]]
No Mask
[[0.49810567]]
No Mask


In [82]:
sample=cv2.imread('sample/3.png')
sample=cv2.resize(sample,(224,224))
detect_mask(sample)

[[0.5029448]]


'No Mask'

In [48]:
X[0].shape

(224, 224, 3)

In [58]:
for category in categories:
    path=os.path.join('test',category)
    label=categories.index(category)
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        print(detect_mask(img))

[[0.50461614]]
No Mask
[[0.50485533]]
No Mask
[[0.50086045]]
No Mask
[[0.50614995]]
No Mask
[[0.50403386]]
No Mask
[[0.5046356]]
No Mask
[[0.50406444]]
No Mask
[[0.5042854]]
No Mask
[[0.5044417]]
No Mask
[[0.5028761]]
No Mask
[[0.503213]]
No Mask
[[0.5040262]]
No Mask
[[0.5041375]]
No Mask
[[0.5040512]]
No Mask
[[0.50210035]]
No Mask
[[0.5036193]]
No Mask
[[0.5025171]]
No Mask
[[0.50380045]]
No Mask
[[0.5049083]]
No Mask
[[0.504629]]
No Mask
[[0.5031721]]
No Mask
[[0.5046594]]
No Mask
[[0.50406194]]
No Mask
[[0.50304]]
No Mask
[[0.50445646]]
No Mask
[[0.5045101]]
No Mask
[[0.5050992]]
No Mask
[[0.50544304]]
No Mask
[[0.50211024]]
No Mask
[[0.503772]]
No Mask
[[0.5040511]]
No Mask
[[0.50494003]]
No Mask
[[0.50452346]]
No Mask
[[0.50421226]]
No Mask
[[0.5029273]]
No Mask
[[0.50393385]]
No Mask
[[0.50368625]]
No Mask
[[0.5032941]]
No Mask
[[0.504768]]
No Mask
[[0.5043349]]
No Mask
[[0.5016636]]
No Mask
[[0.5036324]]
No Mask
[[0.50441015]]
No Mask
[[0.5066923]]
No Mask
[[0.5039887]]
No Mas

[[0.5054706]]
No Mask
[[0.50490713]]
No Mask
[[0.5019891]]
No Mask
[[0.5044704]]
No Mask
[[0.50385416]]
No Mask
[[0.50431585]]
No Mask
[[0.5054648]]
No Mask
[[0.50486046]]
No Mask
[[0.50303596]]
No Mask
[[0.5040243]]
No Mask
[[0.50429773]]
No Mask
[[0.5027392]]
No Mask
[[0.5045885]]
No Mask
[[0.50636834]]
No Mask
[[0.50124437]]
No Mask
[[0.50239027]]
No Mask
[[0.5051305]]
No Mask
[[0.50305384]]
No Mask
[[0.5053126]]
No Mask
[[0.50324386]]
No Mask
[[0.5019843]]
No Mask
[[0.50423825]]
No Mask
[[0.50329196]]
No Mask
[[0.5054659]]
No Mask
[[0.5030173]]
No Mask
[[0.50414044]]
No Mask
[[0.5042958]]
No Mask
[[0.5049742]]
No Mask
[[0.50293016]]
No Mask
[[0.50518036]]
No Mask
[[0.50091875]]
No Mask
[[0.5052087]]
No Mask
[[0.50391155]]
No Mask
[[0.50393635]]
No Mask
[[0.5035106]]
No Mask
[[0.5032154]]
No Mask
[[0.50317466]]
No Mask
[[0.50334316]]
No Mask
[[0.5043737]]
No Mask
[[0.50230694]]
No Mask
[[0.49934033]]
No Mask
[[0.5044533]]
No Mask
[[0.5012736]]
No Mask
[[0.50460446]]
No Mask
[[0.5040

[[0.50450414]]
No Mask
[[0.50325733]]
No Mask
[[0.5039422]]
No Mask
[[0.5058135]]
No Mask
[[0.50350183]]
No Mask
[[0.5033687]]
No Mask
[[0.50366884]]
No Mask
[[0.5026123]]
No Mask
[[0.5049756]]
No Mask
[[0.50462276]]
No Mask
[[0.50376016]]
No Mask
[[0.50476015]]
No Mask
[[0.50439787]]
No Mask
[[0.50289285]]
No Mask
[[0.5034231]]
No Mask
[[0.5029449]]
No Mask
[[0.5046879]]
No Mask
[[0.5008845]]
No Mask
[[0.5048402]]
No Mask
[[0.50305164]]
No Mask
[[0.50405943]]
No Mask
[[0.5031905]]
No Mask
[[0.50402045]]
No Mask
[[0.50526714]]
No Mask
[[0.5052141]]
No Mask
[[0.5027625]]
No Mask
[[0.50373703]]
No Mask
[[0.50507313]]
No Mask
[[0.50464267]]
No Mask
[[0.5033505]]
No Mask
[[0.5042884]]
No Mask
[[0.5048071]]
No Mask
[[0.5046936]]
No Mask
[[0.5047679]]
No Mask
[[0.5038144]]
No Mask
[[0.5028301]]
No Mask
[[0.5017004]]
No Mask
[[0.50466526]]
No Mask
[[0.50374204]]
No Mask
[[0.50266004]]
No Mask
[[0.50465477]]
No Mask
[[0.50414205]]
No Mask
[[0.50319207]]
No Mask
[[0.50465363]]
No Mask
[[0.50397

[[0.5042207]]
No Mask
[[0.5022506]]
No Mask
[[0.50275594]]
No Mask
[[0.5040541]]
No Mask
[[0.5041511]]
No Mask
[[0.5034569]]
No Mask
[[0.50286424]]
No Mask
[[0.50326246]]
No Mask
[[0.5050333]]
No Mask
[[0.5036042]]
No Mask
[[0.5051402]]
No Mask
[[0.50484705]]
No Mask
[[0.503591]]
No Mask
[[0.5045696]]
No Mask
[[0.50432867]]
No Mask
[[0.5046051]]
No Mask
[[0.5049746]]
No Mask
[[0.5038124]]
No Mask
[[0.503595]]
No Mask
[[0.5049742]]
No Mask
[[0.5057172]]
No Mask
[[0.5042834]]
No Mask
[[0.5047125]]
No Mask
[[0.5049703]]
No Mask
[[0.50296384]]
No Mask
[[0.50366884]]
No Mask
[[0.5036754]]
No Mask
[[0.50504255]]
No Mask
[[0.5048607]]
No Mask
[[0.50059855]]
No Mask
[[0.50214076]]
No Mask
[[0.5048164]]
No Mask
[[0.50355333]]
No Mask
[[0.5041193]]
No Mask
[[0.5045461]]
No Mask
[[0.50463486]]
No Mask
[[0.5033371]]
No Mask
[[0.50406045]]
No Mask
[[0.50420016]]
No Mask
[[0.5011335]]
No Mask
[[0.50359595]]
No Mask
[[0.50330997]]
No Mask
[[0.5050101]]
No Mask
[[0.5043986]]
No Mask
[[0.5036266]]
No M

# Fitting on only one class

In [67]:
data_new=[]

In [68]:
path=os.path.join('train_new','with Mask')
label=categories.index('with Mask')
for file in os.listdir(path):
    img_path=os.path.join(path,file)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    data_new.append([img,label])

In [72]:
len(data_new)

2

In [69]:
X_new=[]
y_new=[]
for features,label in data_new:
    X_new.append(features)
    y_new.append(label)

In [73]:
len(X_new)

2

In [74]:
X_new=np.array(X_new)
y_new=np.array(y_new)
X_new=X_new/255

In [76]:
X_train_new,X_test_new,y_train_new,y_test_new=train_test_split(X_new,y_new,test_size=0.5,random_state=42)

In [83]:
model.fit(X_train_new,y_train_new,epochs=10,validation_data=(X_test_new,y_test_new))

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.6928 - accuracy: 1.0000 - val_loss: 0.6920 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 1s 973ms/step - loss: 0.6920 - accuracy: 1.0000 - val_loss: 0.6911 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 1s 934ms/step - loss: 0.6911 - accuracy: 1.0000 - val_loss: 0.6901 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 1s 946ms/step - loss: 0.6901 - accuracy: 1.0000 - val_loss: 0.6891 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 1s 954ms/step - loss: 0.6891 - accuracy: 1.0000 - val_loss: 0.6880 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 1s 928ms/step - loss: 0.6880 - accuracy: 1.0000 - val_loss: 0.6869 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 1s 967ms/step - loss: 0.6869 - accuracy: 1.0000 - val_loss: 0.6858 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=======

In [85]:
sample2=cv2.imread('sample/2.jpg')
sample2=cv2.resize(sample1,(224,224))
detect_mask(sample2)

[[0.49835473]]


'No Mask'